In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [ ]:
def getRawFeatures(picture):
    red = []
    green = []
    blue = []
    for row in range(picture.shape[0]):
        for col in range(picture.shape[1]):
            red.append(picture[row][col][0])
            green.append(picture[row][col][1])
            blue.append(picture[row][col][2])
    feature = red
    feature.extend(green)
    feature.extend(blue)
    return feature

In [ ]:
import io
import bson # this is installed with the pymongo package
import matplotlib
import matplotlib.pyplot as plt
from skimage.data import imread   # or, whatever image library you prefer
import multiprocessing as mp      # will come in handy due to the size of the data
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
# Simple data processing
count_images = 0
image_names_array = []
category_id_array = []
data = bson.decode_file_iter(open('../input/train_example.bson', 'rb'))
pictures = []
count = 0
prod_to_category = dict()

for c, d in enumerate(data):
    #for each product_id
    product_id = d['_id']
    category_id = d['category_id'] # This won't be in Test data
    prod_to_category[product_id] = category_id
    
    for e, pic in enumerate(d['imgs']):
        #for each image
        picture = imread(io.BytesIO(pic['picture']))
        pictures.append(picture)
        count = count + 1
        # do something with the picture, etc
#         image_name = "prod_id-" + str(product_id) + "-" + "image-" + str(e)
#         print("PRODUCT ID:", product_id, "NUMBER", e)
#         plt.imshow(picture)
#         fig1 = plt.gcf()
#         plt.show()
#         plt.draw()
        count_images = count_images + 1
#         image_names_array.append(image_name)
        category_id_array.append(str(category_id))
        #fig1.savefig("img/" + str(image_name), dpi=100)
    print("done")
#     break

prod_to_category = pd.DataFrame.from_dict(prod_to_category, orient='index')
prod_to_category.index.name = '_id'
prod_to_category.rename(columns={0: 'category_id'}, inplace=True)

In [ ]:
X_train = np.asarray(pictures)
y_train = np.asarray(category_id_array)
y_train.shape

In [ ]:
X_train.shape

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 3, 180, 180).astype('float32')
X_train = X_train - np.mean(X_train) / X_train.std()

In [ ]:
# listFeatureVector = []

# for picture in pictures:
#     featureVector = getRawFeatures(picture)
#     listFeatureVector.append(featureVector)

In [ ]:
# print(len(listFeatureVector[0]))

# X = listFeatureVector
# y = category_id_array

In [ ]:
# len(list(set(y)))

In [ ]:
y_train

In [ ]:
b,c = np.unique(y_train, return_inverse=True)

In [ ]:
from collections import Counter
d = Counter(c)

In [ ]:
y_train = c

In [ ]:
y_train

In [ ]:
from keras.utils import np_utils
from tflearn.data_utils import to_categorical
y_train = np_utils.to_categorical(y_train)

In [ ]:
y_train

In [ ]:
# X_train = X_train.reshape(X_train.shape[0], 180, 180, 3)
# X_train = X_train.astype('float32')
# X_train = X_train/255

In [ ]:
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
model = input_data(shape=[None,3,180,180])
model = conv_2d(model,32,5,activation='elu')
model = max_pool_2d(model,2)
model = conv_2d(model,64,3,activation='relu')
model = max_pool_2d(model,2)
model = dropout(model,0.3)
model = conv_2d(model,64,3, activation='elu')
model = max_pool_2d(model,2)
model = fully_connected(model,512,activation='sigmoid')
model = dropout(model,0.3)
model = fully_connected(model,36,activation='softmax')
model = regression(model,optimizer='adagrad',loss='categorical_crossentropy',learning_rate=0.05)

In [ ]:
import tensorflow as tf
import tflearn
with tf.device('cpu:0'):
   model = tflearn.DNN(model)
   model.fit(X_train , y_train)

In [ ]:
pred = model.predict(X_train)

In [ ]:
model.evaluate(X_train, y_train)

In [ ]:
# from keras.preprocessing import image
# img = image.load_img('',target_size=(180,180))
# img = image.img_to_array(img)
# img = np.expand_dims(img, axis=0)
img = X_train[0].reshape(1,3,180,180)
# img.shape
preds = model.predict(img)
preds

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(180,180,3)))
BatchNormalization(axis=-1)
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
BatchNormalization(axis=-1)
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3, 3)))
BatchNormalization(axis=-1)
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
BatchNormalization(axis=-1)
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

# Fully connected layer
model.add(Dense(512))

BatchNormalization()
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(110))

model.add(Activation('softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                         height_shift_range=0.08, zoom_range=0.08)


In [ ]:
train_generator = gen.flow(X_train, y_train)


In [ ]:
model.fit_generator(train_generator, steps_per_epoch=110//64, epochs=2)